In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from IPython.display import display, Markdown

Pour un groupe de pays et de produits donnés, <br>
Ce notebook calcul automatiquement différentes statistiques.

<a href="#parametres" class="btn btn-success" >
**PARAMETRES A ENTRER**
</a>
<br> <br>
<a href="#resultat" class="btn btn-success" >
**AFFICHAGE DES RESULTATS**
</a>

connexion avec la base de donnée

In [2]:
bdd_path = '../3_base_de_donnee/'
bdd_engine = create_engine(f'sqlite:///{bdd_path}donnees_alimentaires.sqlite3')

## fonctions utilisées

In [3]:
# Cette fonction transfrome un array de strings
# en une chaine que l'on peut concatener dans une requête SQL
# au sein d'un IN, ALL
def sql(array):
    return ','.join([f"\"{item}\"" for item in array])

## set de pays, produits

In [4]:
# produit ordonnée par kcal/kg
liste = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT produit, nature,  AVG(dispo_alim_kcal / dispo_alim_kg) AS kcal_kg
FROM(
	SELECT d.nature, d.annee, d.pays, d.produit, d.dispo_alim_tonnes * 1000 AS dispo_alim_kg, 
		   dispo_alim_kcal_p_j * 365 * p.population AS dispo_alim_kcal
	FROM dispo_alim AS d
		JOIN population AS p
			ON d.code_pays = p.code_pays AND d.annee = p.annee
	ORDER BY d.pays, d.produit
)
GROUP BY produit
HAVING kcal_kg IS NOT NULL
ORDER BY kcal_kg
""")

In [5]:
# produits ordonnés par disponibilité alimentaire mondiale en kcal
liste = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT  produit,
		nature,
		SUM(dispo_alim_kcal_p_j * 365 * p.population) AS dispo_alim_kcal
FROM dispo_alim AS d
	JOIN population AS p
		ON d.code_pays = p.code_pays AND d.annee = p.annee
WHERE d.annee = 2013 AND nature='animal'
GROUP BY produit
ORDER BY dispo_alim_kcal ASC
""")
# for i in liste.produit:
#     print(f"\"{i}\"", end=', ')

In [6]:
# quelles sont les produits les plus utilisé pour nourrir les animaux dans le monde ?
liste = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT produit, alim_ani FROM
	(SELECT produit, sum(alim_ani) as alim_ani
	from equilibre_prod
	where alim_ani IS NOT NULL
	GROUP BY produit
	ORDER BY alim_ani DESC
)
""")
# for i in liste.produit:
#     print(f"\"{i}\"", end=', ') 
liste['prop'] = liste.alim_ani / liste.alim_ani.sum() * 100

In [7]:
# ce sont les produits les plus exporté par les 30 pays les plus sous-alimenté
# liste = pd.read_sql_query(con=bdd_engine, sql=f"""
# SELECT produit, SUM(exportation) as export 
# FROM stock 
# WHERE 
#             pays IN({sql(PAYS_SS_ALIM_ASC[-30:])})
#         AND annee={ANNEE}
# GROUP BY produit
# ORDER BY export DESC
# """)
# for i in liste.produit:
#     print(f"\"{i}\"", end=', ') 

#### produits

In [8]:
# tous les produits dans un ordre aléatoire
PRODUITS_TOUS = ["Blé", "Orge", "Maïs", "Seigle", "Avoine", "Millet", "Sorgho", "Céréales, Autres", "Pommes de Terre", "Manioc", "Racines, Nda", "Sucre, betterave", "Sucre Eq Brut", "Edulcorants Autres", "Haricots", "Pois", "Légumineuses Autres", "Noix", "Soja", "Arachides Decortiquees", "Graines de tournesol", "Graines Colza/Moutarde", "Coco (Incl Coprah)", "Sésame", "Olives", "Plantes Oleiferes, Autre", "Huile de Soja", "Huile d'Arachide", "Huile de Tournesol", "Huile de Colza&Moutarde", "Huile Graines de Coton", "Huile de Palmistes", "Huile de Palme", "Huile de Coco", "Huile de Sésame", "Huile d'Olive", "Huile de Germe de Maïs", "Huil Plantes Oleif Autr", "Tomates", "Oignons", "Légumes, Autres", "Oranges, Mandarines", "Citrons & Limes", "Pamplemousse", "Agrumes, Autres", "Bananes", "Bananes plantains", "Pommes", "Ananas", "Dattes", "Raisin", "Fruits, Autres", "Café", "Feve de Cacao", "Thé", "Poivre", "Piments", "Girofles", "Épices, Autres", "Vin", "Bière", "Boissons Fermentés", "Boissons Alcooliques", "Alcool, non Comestible", "Aliments pour enfants", "Viande de Bovins", "Viande d'Ovins/Caprins", "Viande de Suides", "Viande de Volailles", "Viande, Autre", "Abats Comestible", "Graisses Animales Crue", "Beurre, Ghee", "Crème", "Oeufs", "Miel", "Poissons Eau Douce", "Perciform", "Poissons Pelagiques", "Poissons Marins, Autres", "Crustacés", "Cephalopodes", "Mollusques, Autres", "Animaux Aquatiques Autre", "Plantes Aquatiques", "Huiles de Poissons", "Huiles de Foie de Poisso", "Riz (Eq Blanchi)", "Lait - Excl Beurre", "Miscellanees", "Sucre, canne", "Graines de coton", "Patates douces", "Palmistes", "Ignames", "Sucre non centrifugé", "Huile de Son de Riz", "Viande de Anim Aquatiq", ]

# les produits ordonnées du moins calorique au plus calorique (kcal / kg)
PRODUITS_CAL_KG_ASC = ["Animaux Aquatiques Autre", "Mollusques, Autres", "Agrumes, Autres", "Pamplemousse", "Citrons & Limes", "Thé", "Tomates", "Sucre, canne", "Oranges, Mandarines", "Boissons Fermentés", "Légumes, Autres", "Crustacés", "Pommes", "Ananas", "Sucre, betterave", "Café", "Cephalopodes", "Oignons", "Plantes Aquatiques", "Fruits, Autres", "Bière", "Girofles", "Perciform", "Raisin", "Vin", "Bananes", "Lait - Excl Beurre", "Poissons Eau Douce", "Poissons Marins, Autres", "Pommes de Terre", "Bananes plantains", "Racines, Nda", "Olives", "Ignames", "Patates douces", "Manioc", "Poissons Pelagiques", "Viande, Autre", "Abats Comestible", "Coco (Incl Coprah)", "Dattes", "Crème", "Viande de Volailles", "Oeufs", "Viande de Bovins", "Soja", "Poivre", "Avoine", "Viande d'Ovins/Caprins", "Orge", "Graines de tournesol", "Seigle", "Céréales, Autres", "Viande de Suides", "Miel", "Edulcorants Autres", "Noix", "Piments", "Huiles de Foie de Poisso", "Boissons Alcooliques", "Plantes Oleiferes, Autre", "Blé", "Maïs", "Sorgho", "Millet", "Pois", "Aliments pour enfants", "Légumineuses Autres", "Épices, Autres", "Sucre non centrifugé", "Haricots", "Graines Colza/Moutarde", "Sucre Eq Brut", "Riz (Eq Blanchi)", "Feve de Cacao", "Sésame", "Arachides Decortiquees", "Palmistes", "Beurre, Ghee", "Huiles de Poissons", "Graisses Animales Crue", "Huile d'Olive", "Huile de Sésame", "Huile de Coco", "Huile de Palmistes", "Huile d'Arachide", "Huile Graines de Coton", "Huile de Germe de Maïs", "Huil Plantes Oleif Autr", "Huile de Tournesol", "Huile de Colza&Moutarde", "Huile de Palme", "Huile de Soja", "Huile de Son de Riz"]
PRODUITS_ANIMAL_CAL_KG_ASC = ["Animaux Aquatiques Autre", "Mollusques, Autres", "Crustacés", "Cephalopodes", "Plantes Aquatiques", "Perciform", "Lait - Excl Beurre", "Poissons Eau Douce", "Poissons Marins, Autres", "Poissons Pelagiques", "Viande, Autre", "Abats Comestible", "Crème", "Viande de Volailles", "Oeufs", "Viande de Bovins", "Viande d'Ovins/Caprins", "Viande de Suides", "Huiles de Foie de Poisso", "Beurre, Ghee", "Huiles de Poissons", "Graisses Animales Crue"]
PRODUIT_VEGETAL_CAL_KG_ASC = ["Agrumes, Autres", "Pamplemousse", "Citrons & Limes", "Thé", "Tomates", "Sucre, canne", "Oranges, Mandarines", "Boissons Fermentés", "Légumes, Autres", "Pommes", "Ananas", "Sucre, betterave", "Café", "Oignons", "Fruits, Autres", "Bière", "Girofles", "Raisin", "Vin", "Bananes", "Pommes de Terre", "Bananes plantains", "Racines, Nda", "Olives", "Ignames", "Patates douces", "Manioc", "Coco (Incl Coprah)", "Dattes", "Soja", "Poivre", "Avoine", "Orge", "Graines de tournesol", "Seigle", "Céréales, Autres", "Miel", "Edulcorants Autres", "Noix", "Piments", "Boissons Alcooliques", "Plantes Oleiferes, Autre", "Blé", "Maïs", "Sorgho", "Millet", "Pois", "Aliments pour enfants", "Légumineuses Autres", "Épices, Autres", "Sucre non centrifugé", "Haricots", "Graines Colza/Moutarde", "Sucre Eq Brut", "Riz (Eq Blanchi)", "Feve de Cacao", "Sésame", "Arachides Decortiquees", "Palmistes", "Huile d'Olive", "Huile de Sésame", "Huile de Coco", "Huile de Palmistes", "Huile d'Arachide", "Huile Graines de Coton", "Huile de Germe de Maïs", "Huil Plantes Oleif Autr", "Huile de Tournesol", "Huile de Colza&Moutarde", "Huile de Palme", "Huile de Soja", "Huile de Son de Riz"]

# les produits ordonnés par disponibilité calorique mondiale (de la + petite a la + grande)
PRODUIT_DISP_CAL_MONDE = ["Graines de coton", "Viande de Anim Aquatiq", "Sucre, betterave", "Palmistes", "Girofles", "Huiles de Foie de Poisso", "Animaux Aquatiques Autre", "Huiles de Poissons", "Poivre", "Graines de tournesol", "Pamplemousse", "Thé", "Cephalopodes", "Agrumes, Autres", "Miscellanees", "Citrons & Limes", "Aliments pour enfants", "Mollusques, Autres", "Plantes Oleiferes, Autre", "Olives", "Café", "Graines Colza/Moutarde", "Plantes Aquatiques", "Miel", "Crustacés", "Crème", "Poissons Marins, Autres", "Huile de Sésame", "Ananas", "Huile de Son de Riz", "Avoine", "Viande, Autre", "Sésame", "Piments", "Sucre, canne", "Dattes", "Feve de Cacao", "Boissons Fermentés", "Perciform", "Huil Plantes Oleif Autr", "Racines, Nda", "Seigle", "Vin", "Céréales, Autres", "Épices, Autres", "Huile de Palmistes", "Abats Comestible", "Raisin", "Huile de Coco", "Orge", "Huile de Germe de Maïs", "Pois", "Bananes plantains", "Poissons Pelagiques", "Sucre non centrifugé", "Oranges, Mandarines", "Huile d'Olive", "Tomates", "Viande d'Ovins/Caprins", "Oignons", "Pommes", "Coco (Incl Coprah)", "Ignames", "Poissons Eau Douce", "Huile d'Arachide", "Huile Graines de Coton", "Soja", "Noix", "Bananes", "Patates douces", "Haricots", "Edulcorants Autres", "Boissons Alcooliques", "Arachides Decortiquees", "Millet", "Sorgho", "Graisses Animales Crue", "Beurre, Ghee", "Fruits, Autres", "Bière", "Huile de Colza&Moutarde", "Huile de Tournesol", "Oeufs", "Légumineuses Autres", "Manioc", "Viande de Bovins", "Huile de Palme", "Viande de Volailles", "Pommes de Terre", "Légumes, Autres", "Huile de Soja", "Viande de Suides", "Lait - Excl Beurre", "Maïs", "Sucre Eq Brut", "Blé", "Riz (Eq Blanchi)"]
PRODUIT_VEGETAL_DISP_CAL_MONDE = ["Graines de coton", "Sucre, betterave", "Palmistes", "Girofles", "Poivre", "Graines de tournesol", "Pamplemousse", "Thé", "Agrumes, Autres", "Miscellanees", "Citrons & Limes", "Aliments pour enfants", "Plantes Oleiferes, Autre", "Olives", "Café", "Graines Colza/Moutarde", "Miel", "Huile de Sésame", "Ananas", "Huile de Son de Riz", "Avoine", "Sésame", "Piments", "Sucre, canne", "Dattes", "Feve de Cacao", "Boissons Fermentés", "Huil Plantes Oleif Autr", "Racines, Nda", "Seigle", "Vin", "Céréales, Autres", "Épices, Autres", "Huile de Palmistes", "Raisin", "Huile de Coco", "Orge", "Huile de Germe de Maïs", "Pois", "Bananes plantains", "Sucre non centrifugé", "Oranges, Mandarines", "Huile d'Olive", "Tomates", "Oignons", "Pommes", "Coco (Incl Coprah)", "Ignames", "Huile d'Arachide", "Huile Graines de Coton", "Soja", "Noix", "Bananes", "Patates douces", "Haricots", "Edulcorants Autres", "Boissons Alcooliques", "Arachides Decortiquees", "Millet", "Sorgho", "Fruits, Autres", "Bière", "Huile de Colza&Moutarde", "Huile de Tournesol", "Légumineuses Autres", "Manioc", "Huile de Palme", "Pommes de Terre", "Légumes, Autres", "Huile de Soja", "Maïs", "Sucre Eq Brut", "Blé", "Riz (Eq Blanchi)"]
PRODUIT_ANIMAL_DISP_CAL_MONDE = ["Viande de Anim Aquatiq", "Huiles de Foie de Poisso", "Animaux Aquatiques Autre", "Huiles de Poissons", "Cephalopodes", "Mollusques, Autres", "Plantes Aquatiques", "Crustacés", "Crème", "Poissons Marins, Autres", "Viande, Autre", "Perciform", "Abats Comestible", "Poissons Pelagiques", "Viande d'Ovins/Caprins", "Poissons Eau Douce", "Graisses Animales Crue", "Beurre, Ghee", "Oeufs", "Viande de Bovins", "Viande de Volailles", "Viande de Suides", "Lait - Excl Beurre"]

# pour nourrir les animaux dans le monde, on utilise 41% de maïs et 10% de blé
PRODUITS_ALIM_ANIM = ['Maïs', 'Blé']

# ce sont les produits les plus exporté par les 30 pays les plus sous-alimenté
PRODUITS_EXPORT_PAR_PAYS_SS_ALIM_DESC = ["Poissons Pelagiques", "Sucre Eq Brut", "Perciform", "Blé", "Maïs", "Café", "Oranges, Mandarines", "Bière", "Bananes", "Lait - Excl Beurre", "Légumes, Autres", "Noix", "Fruits, Autres", "Riz (Eq Blanchi)", "Oignons", "Pois", "Dattes", "Sésame", "Thé", "Poissons Eau Douce", "Graines de coton", "Huile de Palme", "Coco (Incl Coprah)", "Haricots", "Légumineuses Autres", "Feve de Cacao", "Manioc", "Mollusques, Autres", "Huiles de Poissons", "Sorgho", "Crustacés", "Arachides Decortiquees", "Boissons Alcooliques", "Poissons Marins, Autres", "Viande de Bovins", "Agrumes, Autres", "Orge", "Plantes Oleiferes, Autre", "Céréales, Autres", "Huile de Tournesol", "Pommes de Terre", "Graines de tournesol", "Plantes Aquatiques", "Alcool, non Comestible", "Boissons Fermentés", "Pommes", "Huile de Coco", "Raisin", "Viande d'Ovins/Caprins", "Girofles", "Cephalopodes", "Huil Plantes Oleif Autr", "Huile de Sésame", "Pamplemousse", "Vin", "Citrons & Limes", "Huile d'Olive", "Soja", "Abats Comestible", "Edulcorants Autres", "Graisses Animales Crue", "Huile Graines de Coton", "Tomates", "Huile de Soja", "Racines, Nda", "Viande, Autre", "Épices, Autres", "Beurre, Ghee", "Ananas", "Huiles de Foie de Poisso", "Oeufs", "Animaux Aquatiques Autre", "Huile de Palmistes", "Millet", "Patates douces", "Poivre", "Viande de Volailles", "Bananes plantains", "Miel", "Viande de Suides", "Aliments pour enfants", "Avoine", "Crème", "Graines Colza/Moutarde", "Huile d'Arachide", "Huile de Colza&Moutarde", "Huile de Germe de Maïs", "Ignames", "Olives", "Palmistes", "Piments", "Seigle", "Sucre non centrifugé", "Sucre, betterave", "Sucre, canne", "Huile de Son de Riz"]

TOP_20_IMPORT_HUILE_PALME = ["Inde", "Chine, continentale", "Pays-Bas", "Allemagne", "Pakistan", "Italie", "États-Unis d'Amérique", "Bangladesh", "Espagne", "Malaisie", "Iran (République islamique d')", "Nigéria", "Fédération de Russie", "Belgique", "Égypte", "Japon", "Viet Nam", "France", "Turquie", "République de Corée"]

#### pays

In [9]:
# tout les pays ordonnés du plus 'sous alimenté', au moins 'sous alimenté'
PAYS_SS_ALIM_ASC = ["Danemark", "Finlande", "Norvège", "Hongrie", "Suède", "Azerbaïdjan", "Bélarus", "Irlande", "Nouvelle-Zélande", "Roumanie", "Croatie", "Autriche", "Pays-Bas", "Allemagne", "Ukraine", "Kazakhstan", "Italie", "États-Unis d'Amérique", "Suisse", "France", "Japon", "Fédération de Russie", "Royaume-Uni", "Turquie", "Brésil", "Espagne", "Canada", "Australie", "Israël", "Bosnie-Herzégovine", "Pologne", "République de Corée", "Cuba", "Grèce", "Belgique", "Chine - RAS de Hong-Kong", "Tchéquie (la)", "Portugal", "Uruguay", "Koweït", "Lituanie", "Arménie", "Malaisie", "Chili", "Argentine", "Slovaquie", "Bulgarie", "Maroc", "Émirats arabes unis", "Chine, Taiwan Province de", "Mexique", "Tunisie", "Algérie", "l'ex-République yougoslave de Macédoine", "Égypte", "Slovénie", "Lettonie", "Afrique du Sud", "Serbie", "Iran (République islamique d')", "Oman", "Arabie saoudite", "Turkménistan", "Venezuela (République bolivarienne du)", "Costa Rica", "Albanie", "Mali", "Ghana", "Géorgie", "Nigéria", "Cameroun", "Kirghizistan", "Trinité-et-Tobago", "Ouzbékistan", "Indonésie", "Mauritanie", "Estonie", "Maurice", "Équateur", "Colombie", "Chypre", "Thaïlande", "Pérou", "Népal", "Bénin", "Chine, continentale", "Panama", "Liban", "Niger", "Jamaïque", "Gambie", "El Salvador", "Sri Lanka", "Fidji", "Paraguay", "Gabon", "Viet Nam", "Myanmar", "Jordanie", "République dominicaine", "Guyana", "Sénégal", "Philippines", "Lesotho", "Honduras", "Monténégro", "Guinée", "Guatemala", "Bangladesh", "Inde", "Mongolie", "Chine - RAS de Macao", "République démocratique populaire lao", "Îles Salomon", "Nicaragua", "Cambodge", "Suriname", "Luxembourg", "Togo", "Cabo Verde", "Bolivie (État plurinational de)", "Burkina Faso", "Kenya", "Pakistan", "Malawi", "Djibouti", "Côte d'Ivoire", "Malte", "Guinée-Bissau", "Brunéi Darussalam", "Eswatini", "Soudan", "Botswana", "Timor-Leste", "Afghanistan", "Éthiopie", "Iraq", "Sierra Leone", "Maldives", "Yémen", "Rwanda", "Mozambique", "Belize", "Islande", "Namibie", "République-Unie de Tanzanie", "Ouganda", "Barbade", "Madagascar", "Polynésie française", "Angola", "Libéria", "Tchad", "Nouvelle-Calédonie", "Vanuatu", "Congo", "République populaire démocratique de Corée", "République centrafricaine", "Zimbabwe", "Zambie", "Haïti", "Sao Tomé-et-Principe", "Samoa", "Saint-Vincent-et-les Grenadines", "Kiribati", "Dominique"]
# les pays qui ont un nombre de personnes sous alimentés non nulle
PAYS_SS_ALIM = ["Afghanistan", "Afrique du Sud", "Albanie", "Algérie", "Angola", "Arabie saoudite", "Argentine", "Arménie", "Bangladesh", "Bénin", "Bolivie (État plurinational de)", "Botswana", "Bulgarie", "Burkina Faso", "Cambodge", "Cameroun", "Chili", "Chine, continentale", "Chine, Taiwan Province de", "Colombie", "Congo", "Costa Rica", "Côte d'Ivoire", "Djibouti", "Égypte", "El Salvador", "Émirats arabes unis", "Équateur", "Eswatini", "Éthiopie", "Gabon", "Gambie", "Géorgie", "Ghana", "Guatemala", "Guinée", "Guinée-Bissau", "Haïti", "Honduras", "Inde", "Indonésie", "Iran (République islamique d')", "Iraq", "Jamaïque", "Jordanie", "Kenya", "Kirghizistan", "Lesotho", "Liban", "Libéria", "Madagascar", "Malaisie", "Malawi", "Mali", "Maroc", "Mauritanie", "Mexique", "Mongolie", "Mozambique", "Myanmar", "Namibie", "Népal", "Nicaragua", "Niger", "Nigéria", "Oman", "Ouganda", "Ouzbékistan", "Pakistan", "Panama", "Paraguay", "Pérou", "Philippines", "République centrafricaine", "République démocratique populaire lao", "République dominicaine", "République populaire démocratique de Corée", "République-Unie de Tanzanie", "Rwanda", "Sénégal", "Serbie", "Sierra Leone", "Slovaquie", "Soudan", "Sri Lanka", "Tchad", "Thaïlande", "Timor-Leste", "Togo", "Trinité-et-Tobago", "Tunisie", "Turkménistan", "Venezuela (République bolivarienne du)", "Viet Nam", "Yémen", "Zambie", "Zimbabwe"]

<hr style="height:2px;margin:30px">

## Paramètres <span id="parametres"> </span>

In [10]:
ANNEE = 2013 # 2012 ou 2013

PAYS = PAYS_SS_ALIM_ASC[:30]
PRODUITS = ["Huile de Palme"]

Un fois que les paramètres sont remplis, il est nécessaire de clicker sur le bouton : <br>
## **`restart the kernel and re-run the whole notebook`**

<hr style="height:2px;margin:30px">

## Calcul des statistiques

### Population en sous-nutrition et disponibilité alimentaire

#### population et sous-nutrition

In [11]:
population = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT sum(population)
FROM population
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
""").iloc[0,0]
population = 0 if population is None else population

pop_sous_alim = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT sum(nb_personnes)
FROM sous_nutrition
WHERE 
    pays IN({sql(PAYS)})
    AND annee={ANNEE}
""").iloc[0,0]
pop_sous_alim = 0 if pop_sous_alim is None else pop_sous_alim

#### disponibilité alimentaire (kcal, protéine et matière grasse)

In [12]:
# d'abord on group by pays, et on fait la SOMME de la dispo alimentaire des produits au sein de chaque pays
# On fait alors la MOYENNE entre les pays
dispo_alim_kcal_p_j = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(dispo_alim_par_pays)
FROM
   (SELECT SUM(dispo_alim_kcal_p_j) AS dispo_alim_par_pays
    FROM dispo_alim
    WHERE 
            pays IN({sql(PAYS)})
        AND annee={ANNEE}
        AND produit IN ({sql(PRODUITS)})
    GROUP BY pays
   )
""").iloc[0,0]
dispo_alim_kcal_p_j = 0 if dispo_alim_kcal_p_j is None else dispo_alim_kcal_p_j

In [13]:
dispo_prot = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(dispo_prot_par_pays)
FROM
   (SELECT SUM(dispo_prot) AS dispo_prot_par_pays
    FROM dispo_alim
    WHERE 
            pays IN({sql(PAYS)})
        AND annee={ANNEE}
        AND produit IN ({sql(PRODUITS)})
    GROUP BY pays
   )
""").iloc[0,0]
dispo_prot = 0 if dispo_prot is None else dispo_prot

In [14]:
display(Markdown(f"""
La population du groupe de pays est de **{population/1e6:.2f} Millions**. <br>
**{pop_sous_alim / population * 100:.2f} %** de la population est sous alimentée.
"""))


La population du groupe de pays est de **1438.25 Millions**. <br>
**0.00 %** de la population est sous alimentée.


In [15]:
dispo_mat_gr = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(dispo_mat_gr_par_pays)
FROM
   (SELECT SUM(dispo_mat_gr) AS dispo_mat_gr_par_pays
    FROM dispo_alim
    WHERE 
            pays IN({sql(PAYS)})
        AND annee={ANNEE}
        AND produit IN ({sql(PRODUITS)})
    GROUP BY pays
   )
""").iloc[0,0]
dispo_mat_gr = 0 if dispo_mat_gr is None else dispo_mat_gr

### Utilisation de la disponibilité interieure

In [16]:
# nourriture / dispo_int
taux_nourriture_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(nourriture / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_nourriture_dispo_int = 0 if taux_nourriture_dispo_int is None else taux_nourriture_dispo_int

In [17]:
display(Markdown(f"""
Pour Chaque pays, les produits sélectionnés fournissent une disponibilité alimentaire.

Voici la moyenne de cette disponibilité entre les pays selectionnés :
- **{dispo_alim_kcal_p_j:.2f} kcal** par personne par jour
- **{dispo_prot:.2f} g de protéines** par personnes par jour
- **{dispo_mat_gr:.2f} g de matière** grasse par personnes par jour
"""))


Pour Chaque pays, les produits sélectionnés fournissent une disponibilité alimentaire.

Voici la moyenne de cette disponibilité entre les pays selectionnés :
- **47.46 kcal** par personne par jour
- **0.03 g de protéines** par personnes par jour
- **5.35 g de matière** grasse par personnes par jour


In [18]:
# pertes / dispo_int
taux_pertes_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(pertes / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_pertes_dispo_int = 0 if taux_pertes_dispo_int is None else taux_pertes_dispo_int

In [19]:
# semences / dispo_int
taux_semences_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(semences / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_semences_dispo_int = 0 if taux_semences_dispo_int is None else taux_semences_dispo_int

In [20]:
# autres_utilisations / dispo_int
taux_autres_utilisations_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(autres_utilisations / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_autres_utilisations_dispo_int = 0 if taux_autres_utilisations_dispo_int is None else taux_autres_utilisations_dispo_int

In [21]:
# alim_ani / dispo_int
taux_alim_ani_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(alim_ani / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_alim_ani_dispo_int = 0 if taux_alim_ani_dispo_int is None else taux_alim_ani_dispo_int

In [22]:
# traitement / dispo_int
taux_traitement_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(traitement / dispo_int * 100)
FROM equilibre_prod
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_traitement_dispo_int = 0 if taux_traitement_dispo_int is None else taux_traitement_dispo_int

<hr>

### Importation et exportation

In [23]:
# production / dispo_int
taux_production_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(production / dispo_int * 100)
FROM stock
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_production_dispo_int = 0 if taux_production_dispo_int is None else taux_production_dispo_int

In [24]:
# importation / dispo_int
taux_importation_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(importation / dispo_int * 100)
FROM stock
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_importation_dispo_int = 0 if taux_importation_dispo_int is None else taux_importation_dispo_int

In [25]:
# exportation / dispo_int

taux_exportation_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(exportation / dispo_int * 100)
FROM stock
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_exportation_dispo_int = 0 if taux_exportation_dispo_int is None else taux_exportation_dispo_int

In [26]:
#variation_stock
taux_variation_stock_dispo_int = pd.read_sql_query(con=bdd_engine, sql=f"""
SELECT AVG(variation_stock / dispo_int * 100)
FROM stock
WHERE 
        pays IN({sql(PAYS)})
    AND annee={ANNEE}
    AND produit IN ({sql(PRODUITS)})
""").iloc[0,0]
taux_variation_stock_dispo_int = 0 if taux_variation_stock_dispo_int is None else taux_variation_stock_dispo_int

<hr style="height:2px;margin:30px"> 

## Affichage des statisiques 

<a id="resultat" href="#parametres" class="btn btn-success" > ALLER AUX PARAMETRES </a>

In [27]:
taux_traitement_dispo_int

0.0206782464846981

In [28]:
display(Markdown(f"""

### Champ d'étude :

**ANNEE**: {ANNEE} <br>
**PAYS**: {', '.join(PAYS)} <br>
**PRODUITS**: {', '.join(PRODUITS)}

### Population en sous-nutrition et disponibilité alimentaire

#### population en sous-nutrition

La population du groupe de pays est de **{population/1e6:.2f} Millions**. <br>
**{pop_sous_alim / population * 100:.2f} %** de la population est sous alimentée.

#### disponibilité alimentaire

Pour Chaque pays, les produits sélectionnés fournissent ensemble une disponibilité alimentaire nationale.

Voici la moyenne (inter-pays)  de cette disponibilité exprimé en cal, protéine et matière grasse :
- **{dispo_alim_kcal_p_j:.2f} kcal** par personne par jour
- **{dispo_prot:.2f} g de protéines** par personnes par jour
- **{dispo_mat_gr:.2f} g de matière** grasse par personnes par jour

### Utilisation de la disponibilité interieure

Parmis les pays/produits séléctionnés :
Voici l'utilisation de la disponibiltié intérieur :
- **{taux_nourriture_dispo_int:.2f}%** pour la nourriture
- **{taux_pertes_dispo_int:.2f}%** perdus
- **{taux_semences_dispo_int:.2f}%** pour les semences
- **{taux_autres_utilisations_dispo_int:.2f}%** dans d'autres utilisations
- **{taux_alim_ani_dispo_int:.2f}%** pour nourrir les animaux
- **{taux_traitement_dispo_int:.2f}%** dans les traitements

*Rappel : Disponibilité intérieure = Nourriture + Pertes + Semences + Autres utilisations + Aliments pour animaux + traitement*

### Importation et exportation

Parmis les pays/produits séléctionnés :

La disponibilté intérieur résulte de :
- **+{taux_production_dispo_int:.2f}%** de production
- **+{taux_importation_dispo_int:.2f}% ** d'importation
- **+{taux_variation_stock_dispo_int:.2f}%** de variation de stocks (?)
- **-{taux_exportation_dispo_int:.2f}%** d'éxportation


*rappel* : Disponibilité intérieure  = Production + Importation - Exportation + evolution stock

-------------------------------

**ATENTION** : Pour L'utilisation de la dispo int et l'import/export, il y a beaucoup de valeurs nulles. <br>
Chaque proportion n'est donc calculée que sur les produits disponibles. <br>
La somme ne fait donc pas forcément 100% et des valeurs peuvent être abérrantes.

<a id="resultat" href="#parametres" class="btn btn-success" > ALLER AUX PARAMETRES </a>

"""))




### Champ d'étude :

**ANNEE**: 2013 <br>
**PAYS**: Danemark, Finlande, Norvège, Hongrie, Suède, Azerbaïdjan, Bélarus, Irlande, Nouvelle-Zélande, Roumanie, Croatie, Autriche, Pays-Bas, Allemagne, Ukraine, Kazakhstan, Italie, États-Unis d'Amérique, Suisse, France, Japon, Fédération de Russie, Royaume-Uni, Turquie, Brésil, Espagne, Canada, Australie, Israël, Bosnie-Herzégovine <br>
**PRODUITS**: Huile de Palme

### Population en sous-nutrition et disponibilité alimentaire

#### population en sous-nutrition

La population du groupe de pays est de **1438.25 Millions**. <br>
**0.00 %** de la population est sous alimentée.

#### disponibilité alimentaire

Pour Chaque pays, les produits sélectionnés fournissent ensemble une disponibilité alimentaire nationale.

Voici la moyenne (inter-pays)  de cette disponibilité exprimé en cal, protéine et matière grasse :
- **47.46 kcal** par personne par jour
- **0.03 g de protéines** par personnes par jour
- **5.35 g de matière** grasse par personnes par jour

### Utilisation de la disponibilité interieure

Parmis les pays/produits séléctionnés :
Voici l'utilisation de la disponibiltié intérieur :
- **32.02%** pour la nourriture
- **0.20%** perdus
- **0.00%** pour les semences
- **80.48%** dans d'autres utilisations
- **0.00%** pour nourrir les animaux
- **0.02%** dans les traitements

*Rappel : Disponibilité intérieure = Nourriture + Pertes + Semences + Autres utilisations + Aliments pour animaux + traitement*

### Importation et exportation

Parmis les pays/produits séléctionnés :

La disponibilté intérieur résulte de :
- **+90.67%** de production
- **+128.93% ** d'importation
- **+-5.43%** de variation de stocks (?)
- **-28.01%** d'éxportation


*rappel* : Disponibilité intérieure  = Production + Importation - Exportation + evolution stock

-------------------------------

**ATENTION** : Pour L'utilisation de la dispo int et l'import/export, il y a beaucoup de valeurs nulles. <br>
Chaque proportion n'est donc calculée que sur les produits disponibles. <br>
La somme ne fait donc pas forcément 100% et des valeurs peuvent être abérrantes.

<a id="resultat" href="#parametres" class="btn btn-success" > ALLER AUX PARAMETRES </a>

